<a href="https://colab.research.google.com/github/baldan555/github-slideshow/blob/master/CapstoneTire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
train_dataset_path = ("/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH//256/train")
validation_dataset_path = ("/content/drive/MyDrive/CAPSTONE BANGKIT PROJECT BISMILLAH/256/test")

In [5]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
BATCH_SIZE = 16


In [24]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.28,
                                  shear_range=0.3,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size= (IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 290 images belonging to 3 classes.


In [25]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(validation_dataset_path,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                         
                                                             class_mode='categorical',
                                                             shuffle=True)

Found 58 images belonging to 3 classes.


In [26]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : kualitas bagus
1 : kualitas buruk
2 : kualitas menengah


In [31]:

# Membuat model Sequential
model = Sequential([

tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 30, 30, 128)     

In [32]:
labels = train_generator.class_indices.keys()

In [33]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_generator, epochs=20, validation_data = validation_generator, verbose = 1)

Epoch 1/20
19/19 [==============================] - 9s 360ms/step - loss: 1.6484 - accuracy: 0.3207 - val_loss: 1.0932 - val_accuracy: 0.4138
Epoch 2/20
19/19 [==============================] - 5s 271ms/step - loss: 1.1027 - accuracy: 0.3931 - val_loss: 1.1074 - val_accuracy: 0.3448
Epoch 3/20
19/19 [==============================] - 6s 297ms/step - loss: 1.0905 - accuracy: 0.3897 - val_loss: 1.4384 - val_accuracy: 0.3448
Epoch 4/20
19/19 [==============================] - 5s 270ms/step - loss: 1.1099 - accuracy: 0.3862 - val_loss: 1.0917 - val_accuracy: 0.3448
Epoch 5/20
19/19 [==============================] - 6s 341ms/step - loss: 1.0879 - accuracy: 0.3828 - val_loss: 1.1254 - val_accuracy: 0.3448
Epoch 6/20
19/19 [==============================] - 5s 280ms/step - loss: 1.0777 - accuracy: 0.4103 - val_loss: 1.0507 - val_accuracy: 0.3448
Epoch 7/20
19/19 [==============================] - 6s 313ms/step - loss: 1.1185 - accuracy: 0.3828 - val_loss: 1.0704 - val_accuracy: 0.3448
Epoch 

In [39]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense


#Import model dasar pre-trained Xception
model_Xception = Xception(weights='imagenet', input_shape = (224,224,3), include_top=False, pooling = 'avg')

#Output model dasar
x = model_Xception.output

#Input ke DNN data kita
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
prediksi = Dense(3, activation='softmax')(x)

#Model akhir yang akan dilatih
model_tl = Model(inputs=model_Xception.input, outputs=prediksi)

#Freeze semua layer pada Xception
for layer in model_Xception.layers:
    layer.trainable = False

#Sambungan model akhir
model_tl.summary()

#Compile modelnya
model_tl.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

#Latih model tsb
model_tl.fit(train_generator, epochs=20, validation_data = validation_generator, verbose = 1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [42]:
#Freeze 100 layer pertama dan Unfreeze layer sisanya (setelah di freeze)

class MyThresholdCallback(tf.keras.callbacks.Callback):
  def __init__(self, threshold):
    super(MyThresholdCallback, self).__init__()
    self.threshold = threshold

  def on_epoch_end(self, epoch, logs=None): 
    val_accuracy = logs["val_accuracy"]
    train_accuracy = logs["accuracy"]
    if val_accuracy > self.threshold and train_accuracy > self.threshold:
      self.model.stop_training = True

callbacks = EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=10)

for layer in model_tl.layers[:100]:
    layer.trainable = False
for layer in model_tl.layers[100:]:
    layer.trainable = True

#Fine tuning (nilai lr kecil)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001,
                 decay = 1e-11,
                 beta_1=0.8,
                 beta_2=0.999,
                 amsgrad=False)

#Recompile model hasil unfreeze
model_tl.compile(optimizer=optimizer,
                 loss='categorical_crossentropy',
                 metrics = ['accuracy'])

#Ringkasan model hasil unfreeze
model_tl.summary()

performa_tl = model_tl.fit(train_generator,
                           callbacks = [callbacks],
                           epochs=50,
                           validation_data = validation_generator,
                           verbose = 1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           